In [16]:
from abs_models import models
from abs_models import utils

ABS_model = models.get_VAE(n_iter=50)

In [17]:
ABS_model

In [18]:
batch = utils.get_batch()

In [19]:
import numpy as np


def predict_on_batch(model, batch, batch_size):
    """Predicts the digits of an MNIST batch."""
    preds = []
    labels = []
    for i in range(batch_size):
        point, label = utils.get_batch()
        labels.append(label[0])
        tensor_point = utils.n2t(point)
        logits = model(tensor_point)[0]
        logits = [x for x in logits]
        pred = np.argmax(logits)
        preds.append(int(pred))
    return preds, labels

In [20]:
preds, labels = predict_on_batch(ABS_model, batch, 5)

In [21]:
preds

In [22]:
labels

In [23]:
import foolbox

if ABS_model.code_base == "tensorflow":
    fmodel = foolbox.models.TensorFlowModel(
        ABS_model.x_input, ABS_model.pre_softmax, (0.0, 1.0), channel_axis=3
    )
elif ABS_model.code_base == "pytorch":
    ABS_model.eval()
    fmodel = foolbox.models.PyTorchModel(
        ABS_model, bounds=(0.0, 1.0), num_classes=10, device=utils.dev()
    )

Boundary Attack

In [24]:
from foolbox import attacks

images, labels = utils.get_batch(bs=1)

In [25]:
attack = attacks.DeepFoolL2Attack(fmodel)
metric = foolbox.distances.MSE
criterion = foolbox.criteria.Misclassification()

In [27]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.imshow(images[0, 0], cmap="gray")
plt.title("original image")
plt.axis("off")
plt.show()

In [28]:
gradient_estimator = foolbox.gradient_estimators.CoordinateWiseGradientEstimator(0.1)
fmodel = foolbox.models.ModelWithEstimatedGradients(fmodel, gradient_estimator)

adversary = foolbox.adversarial.Adversarial(
    fmodel, criterion, images[0], labels[0], distance=metric
)
attack(adversary)

In [29]:
plt.imshow(a.image[0], cmap="gray")
plt.title("adversarial image")
plt.axis("off")
plt.show()
print("Model prediction:", np.argmax(fmodel.predictions(adversary.image)))

In [30]:
from abs_models import models

traditional_model = models.get_CNN()

In [31]:
traditional_model

In [32]:
preds, labels = predict_on_batch(traditional_model, batch, 5)

In [ ]:
preds

In [ ]:
labels

In [38]:
if traditional_model.code_base == "tensorflow":
    fmodel_traditional = foolbox.models.TensorFlowModel(
        traditional_model.x_input,
        traditional_model.pre_softmax,
        (0.0, 1.0),
        channel_axis=3,
    )
elif traditional_model.code_base == "pytorch":
    traditional_model.eval()
    fmodel_traditional = foolbox.models.PyTorchModel(
        traditional_model, bounds=(0.0, 1.0), num_classes=10, device=u.dev()
    )

In [36]:
fmodel_traditional = foolbox.models.ModelWithEstimatedGradients(fmodel_traditional, GE)

adversarial_traditional = foolbox.adversarial.Adversarial(
    fmodel_traditional, criterion, images[0], labels[0], distance=metric
)
attack(adversarial_traditional)

In [37]:
plt.imshow(adversarial_traditional.image[0], cmap="gray")
plt.title("adversarial image")
plt.axis("off")
plt.show()
print(
    "Model prediction:",
    np.argmax(fmodel_traditional.predictions(adversarial_traditional.image)),
)